In [20]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)


In [21]:
df_custos = pd.read_csv("/home/rodrigo/controladoria-industrial/data/raw/custos_lancamentos_raw.csv")
df_producao = pd.read_csv("/home/rodrigo/controladoria-industrial/data/raw/producao_raw.csv")
df_centros = pd.read_csv("/home/rodrigo/controladoria-industrial/data/raw/centros_custo_raw.csv")
df_produtos = pd.read_csv("/home/rodrigo/controladoria-industrial/data/raw/produtos_raw.csv")


In [22]:
df_custos["data"] = pd.to_datetime(df_custos["data"])
df_producao["data"] = pd.to_datetime(df_producao["data"])


In [23]:
df_custos["categoria"] = (
    df_custos["categoria"]
    .str.strip()
    .str.upper()
)

df_custos["tipo_custo"] = (
    df_custos["tipo_custo"]
    .str.strip()
    .str.upper()
)


In [24]:
df_custos["ajuste"] = df_custos["valor"] < 0

df_custos.loc[df_custos["ajuste"], "valor"] = abs(df_custos["valor"])


In [25]:
df_custos["centro_custo"] = df_custos["centro_custo"].fillna("CC999")


In [26]:
centros_validos = set(df_centros["centro_custo"].tolist()) | {"CC999"}

df_custos["centro_valido"] = df_custos["centro_custo"].isin(centros_validos)


In [27]:
df_custos["centro_valido"].value_counts()


centro_valido
True    800
Name: count, dtype: int64

In [28]:
df_custos["ano"] = df_custos["data"].dt.year
df_custos["mes"] = df_custos["data"].dt.month
df_custos["ano_mes"] = df_custos["data"].dt.to_period("M").astype(str)


In [29]:
df_custos = df_custos.merge(
    df_centros,
    on="centro_custo",
    how="left"
)


In [30]:
fato_custos = df_custos[
    [
        "data", "ano", "mes", "ano_mes",
        "centro_custo", "descricao", "tipo",
        "tipo_custo", "categoria",
        "valor", "ajuste"
    ]
].copy()


In [31]:
fato_custos.isna().sum()


data             0
ano              0
mes              0
ano_mes          0
centro_custo     0
descricao       40
tipo            40
tipo_custo       0
categoria        0
valor            0
ajuste           0
dtype: int64

In [32]:
fato_custos.head()


,data,ano,mes,ano_mes,centro_custo,descricao,tipo,tipo_custo,categoria,valor,ajuste
0,2024-02-01,2024,2,2024-02,CC500,Administrativo,Administrativo,DIRETO,MATERIA PRIMA,14739.93,False
1,2024-05-17,2024,5,2024-05,CC200,Produção Linha B,Produtivo,INDIRETO,ENERGIA,19927.67,False
2,2023-03-31,2023,3,2023-03,CC300,Manutenção,Produtivo,INDIRETO,ENERGIA,17983.71,False
3,2024-12-18,2024,12,2024-12,CC200,Produção Linha B,Produtivo,INDIRETO,MAO DE OBRA,17042.10,False
4,2024-05-20,2024,5,2024-05,CC100,Produção Linha A,Produtivo,DIRETO,ENERGIA,15862.11,False


In [33]:
fato_custos.to_csv("/home/rodrigo/controladoria-industrial/data/processed/fato_custos_tratado.csv", index=False)
df_producao.to_csv("/home/rodrigo/controladoria-industrial/data/processed/producao_tratada.csv", index=False)
